In [8]:
import os
import json
from matplotlib import pyplot as plt
import torch
from task import Task, Blackboard
from tqdm import tqdm
import trainer
import importlib

In [ ]:
trainer.precompute_tasks(
    input_dir="data/evaluation", 
    output_dir="precomputed_tasks/evaluation_400",
    augmentation_types=trainer.augmentation_types
)

In [ ]:
task = trainer.load_precomputed_task("precomputed_tasks/evaluation_400/e57337a4_rotate_270.pt")
print(task.task_id, len(task.train_graphs), len(task.test_graphs))